In [414]:
import os
import psycopg2
import datetime

import pandas as pd

from bokeh.io import output_notebook, show, reset_output
from bokeh.plotting import figure
from bokeh.palettes import all_palettes
from bokeh.embed import file_html
from bokeh.resources import CDN
from bokeh.models import DatetimeTickFormatter, Range1d, WheelZoomTool, HoverTool, ColumnDataSource, CustomJS

os.chdir(r'C:\users\barby\downloads')

%matplotlib inline

km_in_a_mile = 1.60934

def readConfig(key):
    config = pd.read_table(r'c:\users\barby\documents\config.txt', header = None)
    config = [c.split('=') for c in config[0]]
    out = [c[1] for c in config if c[0] == key][0]
    return(out)

def dbGetQuery(q):
    pw = readConfig('pw')
    conn_string = "host='kavdb.c9lrodma91yx.us-west-2.rds.amazonaws.com' dbname='kavdb' user='lkavenagh' password='" + pw + "'"
    conn = psycopg2.connect(conn_string)
    conn.autocommit = True
    dat = pd.read_sql(q, conn)
    conn.close()
    return(dat)

In [183]:
reset_output()
output_notebook()

Loading BokehJS ...

In [290]:
dat = dbGetQuery("SELECT * FROM runs.activities")
dat = dat.sort_values('start_date')

dat = dat.append(dat.tail(1)).reset_index(drop = True)
dat.loc[len(dat)-1, 'start_date'] = datetime.datetime.now()
dat.loc[len(dat)-1, 'distance'] = 0

dat['year'] = [c.year for c in dat.start_date]
dat['doy'] = [c.timetuple().tm_yday for c in dat.start_date]

dat['distance_miles'] = [(c/1000)/km_in_a_mile for c in dat.distance]
dat['ytd_distance'] = dat.groupby('year', as_index = False).cumsum()['distance_miles']

In [482]:
target = 800
x = (datetime.datetime.now().date() - datetime.date(today.year, 1, 1)).days + 1
today_target = (target/max(dat.doy)) * x
actual = dat.loc[dat.start_date == max(dat.start_date), 'ytd_distance'].item()

colors = ['red', 'orange', 'blue', 'green', 'cyan', 'magenta', 'black']

p = figure(plot_width=800, plot_height=400, title="YTD miles", tools="pan,wheel_zoom,box_zoom,hover", active_scroll = 'wheel_zoom')
p.x_range=Range1d(datetime.date(1899,12,31),datetime.date(1900,12,31))
dat['generic_date'] = [datetime.date(1900,1,1) + datetime.timedelta(int(c)) for c in dat.doy]
plot_dat = dat[['generic_date', 'year', 'ytd_distance']]
plot_dat = plot_dat.pivot_table(index = 'generic_date', columns = 'year')
plot_dat.columns = plot_dat.columns.droplevel().rename(None)
plot_dat = plot_dat.reset_index()
plot_dat = plot_dat.apply(lambda series: series.loc[:series.last_valid_index()].ffill())
#plot_dat = plot_dat.fillna(method = 'ffill')
#plot_dat = plot_dat.fillna(0)
x = range(len(plot_dat))
y = [(target / len(plot_dat)) * x for x in x]
plot_dat['target'] = y

plot_dat = plot_dat.reset_index(drop = True)
plot_dat.columns = [str(c) for c in plot_dat.columns]

mypalette = all_palettes['Colorblind'][len(plot_dat.columns)-1]

plot_dat['datestr'] = [c.strftime('%d %b') for c in plot_dat['generic_date']]

source = ColumnDataSource(data = plot_dat)

to_plot = plot_dat.columns[1:-1]
for i,l in enumerate(to_plot):
    if l == 'target':
        p.line(x = 'generic_date',
            y = l,
            line_color = 'black',
            line_dash = 'dashed',
            line_width = 2,
            source=source)
    elif i == (len(plot_dat.columns)-2):
        p.line(x = 'generic_date',
            y = l,
            line_color = mypalette[i],
            line_width = 4,
            source=source)
    else:
        p.line(x = 'generic_date',
            y = l,
            line_color = mypalette[i],
            line_width = 2, legend = ' '+str(i+2012),
            source=source)


p.xaxis.formatter = DatetimeTickFormatter(
        hours=["%d"],
        days=["%d %b"],
        months=["%d %b"],
        years=["%d %b"]
    )
p.xaxis.major_label_orientation = 3/4

p.legend.location = "top_left"
p.legend.click_policy="hide"

hover = p.select(dict(type=HoverTool))

hover.tooltips = [
    ("Date", "@datestr"),
    ("YTD miles", "$y{0.0}"),
    ]
hover.mode='mouse'
show(p)

html = file_html(p, CDN, "my plot")
text_file = open(r"C:\Users\barby\Documents\GitHub\runs\ytd_totals.html", "w")
text_file.write(html)
text_file.close()